# Amino Acid Specific Contact Distance Cutoffs

## Motivation: We want to characterize amino acid specific distance distributions over non-redundant PDB dataset. The reason to do such a calculation is to refine current contact defintion, which always uses a single distance cutoff (e.g., 4.5 or 6.0 angstrom). We want to develop amino acid specific cutoffs to account for distinct interaction ranges between e.g. ionizable residues and between non-polar residues.

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader, mmtfWriter
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.utils import traverseStructureHierarchy, ColumnarStructure
from mmtfPyspark import structureViewer
from mmtfPyspark.filters import ContainsGroup, ContainsLProteinChain, PolymerComposition, Resolution 
import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt

In [118]:
def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                  # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = np.array(ia[1:] != ia[:-1])     # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

In [134]:
a = ['A','A','A','B','B','C']
list(range(len(rle(a)[0])))
dm = np.zeros((10, 10))
np.min(dm[0:5,2:5])

0.0

In [135]:
def distmap(structure):
    arrays = ColumnarStructure(structure, firstModelOnly=True)
    x = arrays.get_x_coords()
    y = arrays.get_y_coords()
    z = arrays.get_z_coords()
#    entity_types = arrays.get_entity_types()
#    cb_idx = entity_types == 'PRO'
#    xc = x[cb_idx]
#    yc = y[cb_idx]
#    zc = z[cb_idx]
#    coords = np.swapaxes(np.array([xc,yc,zc]), 0, 1)
    coords = np.swapaxes(np.array([x,y,z]), 0, 1)
    dist_matrix = squareform(pdist(coords), 'euclidean')
    ch = arrays.get_chain_ids()
    resno = arrays.get_group_numbers()
    string = ch + resno
#    unq = np.unique(string)
    inds = rle(string)
    nres = len(inds[0])
    dm = np.zeros((nres, nres))
    for i in range(nres):
        for j in range(nres):
            dm[i,j] = np.min(dist_matrix[inds[1][i]:(inds[1][i]+inds[0][i]), inds[1][j]:(inds[1][j]+inds[0][j])])
            
    return dm  
#    return dist_matrix
#   atom_names = arrays.get_atom_names()
#    noh_idx = (atom_names = 'H*')

**Configure Sparks**

In [4]:
spark = SparkSession.builder.master("local[4]").appName("Hackthon").getOrCreate()
sc = spark.sparkContext

In [5]:
path = "../resources/mmtf_full_sample"

pdb = mmtfReader.read_sequence_file(path, sc)

In [6]:
nr_chains = pdb \
    .filter(Pisces(sequenceIdentity=20, resolution = 1.6)) \
    .flatMap(StructureToPolymerChains()) \
    .filter(Pisces(sequenceIdentity=20, resolution = 1.6)) \
    .filter(ContainsLProteinChain())

In [7]:
#nr_chains.count()

In [8]:
pdbids = nr_chains.keys().collect()[:10]
pdbids

['4WN5.A',
 '4WND.B',
 '4WP9.A',
 '4WPG.A',
 '4WPK.A',
 '4WRI.A',
 '4WSF.A',
 '1GWM.A',
 '1GXU.A',
 '1GY7.B']

In [9]:
structures = nr_chains.filter(lambda x: x[0] in pdbids)

In [10]:
structures.count()

10

In [66]:
#tt=structures.collect()[0]

In [75]:
#tt[1].num_atoms

1660

In [136]:
tt = structures.map(lambda x: distmap(x[1]))

In [ ]:
output = tt.collect()
#help(ColumnarStructure)

In [115]:
output

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
spark.stop()